Entrenar un modelo T5-Sentinel bajo el paradigma Seq2Seq es un reto técnico excelente para tu TFG

# 1. Preparacion del entorno y datos
T5 espera que las etiquetas sean texto. Vamos a mapear tus is_real a los targets del paper:
0 (IA) $\rightarrow$ "positive"
1 (Humano) $\rightarrow$ "negative"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
import json

# 1. Cargar y formatear para Seq2Seq
def prepare_t5_data(jsonl_path):
    try:
        df = pd.read_json(jsonl_path, lines=True)
    except ValueError as e:
        print(f"Error al leer el archivo JSONL: {e}")
        print("Intentando leer las primeras líneas para diagnóstico...")
        with open(jsonl_path, 'r') as f:
            for i, line in enumerate(f):
                print(f"Línea {i+1}: {line.strip()}")
                if i >= 4: # Imprimir solo las primeras 5 líneas
                    break
        raise # Re-lanzar el error después del diagnóstico

    # Mapeo según metodología GPT-Sentinel
    df['target_text'] = df['is_real'].map({0: "positive", 1: "negative"})
    # Prefijo de tarea (estándar en T5)
    df['input_text'] = "classify authenticity: " + df['title'] + " " + df['content']
    return Dataset.from_pandas(df[['input_text', 'target_text']])

dataset = prepare_t5_data('/content/drive/MyDrive/TFG/multimodal_dataset_fixed.jsonl')
dataset = dataset.train_test_split(test_size=0.1)

# 2. Tokenizacion

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-base")

def tokenize_fn(examples):
    # Tokenizar entradas (noticias)
    model_inputs = tokenizer(examples["input_text"], max_length=512, truncation=True, padding="max_length")
    # Tokenizar objetivos (etiquetas de texto)
    labels = tokenizer(examples["target_text"], max_length=10, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize_fn, batched=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

# 3. Configuración de Hiperparámetros

In [ ]:
# Si el batch real es 8, acumulamos 64 pasos (8 * 64 = 512)
training_args = TrainingArguments(
    output_dir="./t5_sentinel_checkpoints",
    num_train_epochs=5,                          # Según Secc 4.2
    learning_rate=5e-4,                          # Según Tabla 2
    per_device_train_batch_size=8,               # Ajuste para GPU T4
    gradient_accumulation_steps=64,              # Para alcanzar el effective batch size de 512
    weight_decay=1e-3,                           # Según Tabla 2
    lr_scheduler_type="cosine",                  # Cosine annealing
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    # predict_with_generate=True,                # Este parámetro ya no es válido aquí
    fp16=True,                                   # Aceleración por hardware
)

model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-base")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    # tokenizer=tokenizer, # Este parámetro ya no es necesario aquí
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/257 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# 4. Ejecución y Evaluación Restringida
En la inferencia de T5-Sentinel, no queremos que el modelo genere poesía, solo "positive" o "negative".

In [ ]:
# Lanzar entrenamiento
trainer.train()

# Evaluación con restricción de vocabulario
import torch

def get_sentinel_prediction(text, model, tokenizer):
    input_ids = tokenizer("classify authenticity: " + text, return_tensors="pt").input_ids.to("cuda")

    # Obtenemos los logits del primer token generado
    with torch.no_grad():
        outputs = model(input_ids=input_ids, decoder_input_ids=torch.tensor([[tokenizer.pad_token_id]]).to("cuda"))
        logits = outputs.logits[:, 0, :] # Logits del primer token de salida

    # Filtramos solo los IDs de "positive" y "negative"
    pos_id = tokenizer.encode("positive")[0]
    neg_id = tokenizer.encode("negative")[0]

    # Comparamos probabilidades relativas
    probs = torch.softmax(logits[:, [pos_id, neg_id]], dim=-1)
    return "IA" if probs[0][0] > probs[0][1] else "Humano"

Epoch,Training Loss,Validation Loss
1,No log,13.280245
2,No log,13.280245
3,No log,13.280245
4,No log,9.628906
5,No log,8.180064


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def evaluate_model(model, tokenizer, dataset):
    predictions = []
    true_labels = []

    # Mapeo inverso para comparar con los resultados de get_sentinel_prediction
    # is_real = 0 (IA) -> target_text = 'positive'
    # is_real = 1 (Humano) -> target_text = 'negative'
    # get_sentinel_prediction: 'IA' (for positive) o 'Humano' (for negative)

    # Es decir, si get_sentinel_prediction devuelve 'IA', esperamos 'positive'.
    # Si devuelve 'Humano', esperamos 'negative'.

    for example in dataset:
        # Asumimos que 'input_text' ya tiene el prefijo 'classify authenticity: '
        # y que 'target_text' es 'positive' o 'negative'

        # Obtener la predicción del modelo
        predicted_label = get_sentinel_prediction(example['input_text'].replace('classify authenticity: ', ''), model, tokenizer)
        predictions.append(predicted_label)
        true_labels.append(example['target_text'])

    # Para calcular las métricas, necesitamos que las etiquetas sean consistentes.
    # Mapearemos 'positive' -> 'IA' y 'negative' -> 'Humano' para que coincidan con la salida del modelo.
    # O mapearemos 'IA' -> 'positive' y 'Humano' -> 'negative' para que coincidan con el target_text.
    # Optaremos por el segundo para que las métricas se interpreten con 'positive'/'negative'

    mapped_predictions = ["positive" if p == "IA" else "negative" for p in predictions]

    print("\n--- Resultados de Evaluación ---")
    print(f"Total de ejemplos en el conjunto de prueba: {len(true_labels)}")

    # Calcular exactitud
    accuracy = accuracy_score(true_labels, mapped_predictions)
    print(f"Accuracy: {accuracy:.4f}")

    # Calcular precisión, recall, f1-score por clase
    # labels=['positive', 'negative'] asegura el orden y las etiquetas
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels, mapped_predictions, average=None, labels=['positive', 'negative'], zero_division=0
    )

    print("\nMétricas por clase:")
    print(f"  Clase 'positive' (IA):")
    print(f"    Precision: {precision[0]:.4f}")
    print(f"    Recall:    {recall[0]:.4f}")
    print(f"    F1-score:  {f1[0]:.4f}")

    print(f"  Clase 'negative' (Humano):")
    print(f"    Precision: {precision[1]:.4f}")
    print(f"    Recall:    {recall[1]:.4f}")
    print(f"    F1-score:  {f1[1]:.4f}")

    # Calcular métricas macro y weighted (para desbalanceo de clases)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        true_labels, mapped_predictions, average='macro', zero_division=0
    )
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        true_labels, mapped_predictions, average='weighted', zero_division=0
    )

    print("\nMétricas promedio:")
    print(f"  Macro Precision: {precision_macro:.4f}")
    print(f"  Macro Recall:    {recall_macro:.4f}")
    print(f"  Macro F1-score:  {f1_macro:.4f}")
    print(f"  Weighted Precision: {precision_weighted:.4f}")
    print(f"  Weighted Recall:    {recall_weighted:.4f}")
    print(f"  Weighted F1-score:  {f1_weighted:.4f}")


# Ejecutar la evaluación
evaluate_model(model, tokenizer, tokenized_dataset["test"])


--- Resultados de Evaluación ---
Total de ejemplos en el conjunto de prueba: 7
Accuracy: 0.5714

Métricas por clase:
  Clase 'positive' (IA):
    Precision: 0.5714
    Recall:    1.0000
    F1-score:  0.7273
  Clase 'negative' (Humano):
    Precision: 0.0000
    Recall:    0.0000
    F1-score:  0.0000

Métricas promedio:
  Macro Precision: 0.2857
  Macro Recall:    0.5000
  Macro F1-score:  0.3636
  Weighted Precision: 0.3265
  Weighted Recall:    0.5714
  Weighted F1-score:  0.4156
